In [21]:
import numpy as np
import pandas as pd
import cvxpy as cp


## Synthetic dataset

In [22]:
mean1 = [-1,1]
mean2 = [1,-1]
cov = [[1, 0], [0, 1]]
x1, y1 = np.random.multivariate_normal(mean1, cov, 100).T
x2, y2 = np.random.multivariate_normal(mean2, cov, 100).T

plt.plot(x1, y1, 'x')
plt.plot(x2, y2, 'x')
plt.axis('equal')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
m = 3 #rows
l = 3 #columns
X ,Y= np.random.multivariate_normal(mean1, cov, 4).T
Z= np.random.multivariate_normal(mean1, cov, (m,1)).T
print(X,Y)
# print(Z)

[-0.27107071 -1.00078154 -0.18763688 -0.80753856] [ 3.12666893 -1.28550992  0.54687685  1.05453515]


## Defining project problem in cvxpy 

In [ ]:
#svm formulation for synthetic 
m = 5 #rows
l = 4 #columns
num_samples =100
X , Y = np.random.multivariate_normal(mean1, cov, num_samples).T  # synthetic dataset

## M : dimension of 
W_ = cp.Variable((num_samples,1))
w_ = cp.Variable()
loss = cp.sum(cp.pos(1 - cp.multiply(Y, X @ W_ - w_)))
# reg = cp.norm(W, 1)
# lambd = cp.Parameter(nonneg=True)
prob = cp.Problem(cp.Minimize(loss/m))
prob.solve()

8.340041576648943

# References
- https://github.com/sdeepaknarayanan/Machine-Learning/blob/master/Assignment%207/HW7_SVM.ipynb  

- https://www.baeldung.com/cs/svm-hard-margin-vs-soft-margin
- https://www.quora.com/How-do-we-select-which-data-points-act-as-support-vectors-for-SVM


# Papers
- https://link.springer.com/content/pdf/10.1007/s10462-017-9611-1.pdf

# Preparing the data

In [ ]:
dataset = pd.read_csv('mnist/mnist_train.csv')
train_data, train_label = dataset.drop(labels = ["label"],axis = 1).to_numpy(), dataset["label"]
trainlabel = dataset.loc[(dataset['label']== 1)  | (dataset['label']== 7) ]['label']
trainlabel.loc[trainlabel == 1] =1
trainlabel.loc[trainlabel == 7] =-1
traindata = dataset.loc[(dataset['label']== 1)  | (dataset['label']== 7) ]
traindata = traindata.drop(labels = ["label"],axis = 1).to_numpy()
trainlabel = trainlabel.to_numpy()
trainlabel= np.reshape(trainlabel, (trainlabel.shape[0],1))

# SVM formulation for mnist classes 1 and 7

In [ ]:
#normal
m = traindata.shape[1]
W = cp.Variable((m,1))
w = cp.Variable()
objective_func = cp.Minimize(0.5*cp.norm(W,p=2)**2)
const = [trainlabel[i]*(traindata[i]@ W+w)>=1 for i in range(traindata.shape[0])]
prob = cp.Problem(objective_func,constraints=const)

# loss = cp.sum(cp.pos(1 - cp.multiply(train_data, input_row @ W - w)))
# prob = cp.Problem(cp.Minimize(loss/m))
prob.solve()

0.0009312605061763449

In [ ]:
#hinge loss
m1 = traindata.shape[1]
W1 = cp.Variable((m,1))
w1 = cp.Variable()
lambd = cp.Parameter(nonneg=True)

## Ideally we will have to try using different values fro "lambda"
## For the sake of testing the code, we have set it to 0.01
## Do we need to have a lambda?
lambd = 0.01 
reg_loss = cp.norm(W1,p=2)**2

#hinge loss
hinge_loss = cp.sum(cp.pos(1-cp.multiply(trainlabel,traindata @ W1 - w1)))



#Objective is to minimize reg_loss and hinge_loss
# objective_func = cp.Minimize(hinge_loss/m + lambd*reg_loss)
prob1 = cp.Problem(cp.Minimize(hinge_loss/m + lambd*reg_loss))
prob1.solve()

1.862521144922498e-05

## Test Data

In [ ]:
dataset_test = pd.read_csv('mnist/mnist_test.csv')
testlabel = dataset.loc[(dataset['label']== 1)  | (dataset['label']== 7) ]['label']
testlabel.loc[testlabel == 1] =1
testlabel.loc[testlabel == 7] =-1
testdata = dataset.loc[(dataset['label']== 1)  | (dataset['label']== 7) ]
testdata = testdata.drop(labels = ["label"],axis = 1).to_numpy()
testlabel = testlabel.to_numpy()
testlabel= np.reshape(testlabel, (testlabel.shape[0],1))

/home/anw/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/anw/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()


In [38]:
res

0

In [37]:
TEST = len(testdata)
sumn = np.sign(testdata.dot(W1.value) - w1.value)
res = (testlabel != sumn).sum()

In [ ]:
 def f(test_input):
        test_val = test_input.dot(W.value) +  w.value
        if test_val < -1:
            test_val -1
        elif test_val > 1:
            test_val = 1
        
        estimated_class = classes.get(f(test_input))
        return estimated_class

In [ ]:
res = []
performance = []
for i in range(testdata.shape[0]):
    result = f(testdata[i])
    res.append(result)

    ## assessing performance
    if result == testlabel[i]:#[0]:
        performance.append[1]
    else:
        performance.append[0]


NameError: name 'self' is not defined

## Testing the classifier

In [ ]:
testdata[8].dot(W.value) +  w.value

array([-9.55024793])

In [ ]:
testlabel[8]

-1

In [ ]:
positive_class =[]
negative_class = []
for i in range(testdata.shape[0]):
    if testdata[i].dot(W.value) +  w.value>1:
        positive_class.append(testdata[i])
    if testdata[i].dot(W.value) +  w.value<-1:
        negative_class.append(testdata[i])
        

In [ ]:
testdata.shape
        

(13007, 784)

In [ ]:
train_error = (np.sign(X.dot(beta_true)) != np.sign(traindata[i].dot(W.value) +  w.value))
test_error[i] = (np.sign(X_test.dot(beta_true) + offset) != np.sign(X_test.dot(beta.value) - v.value)).sum()/TEST

Variable(())

In [ ]:
train_label.shape

(60000,)